In [27]:
import pandas as pd
import requests
import re
import numpy as np
from bs4 import BeautifulSoup as bs

# 1 - Carga del dataframe de donde se va a usar la URL

In [2]:
df_races = pd.read_csv("./data/races.csv")
df_races

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,1140,2024,20,32,Mexico City Grand Prix,2024-10-27,20:00:00,https://en.wikipedia.org/wiki/2024_Mexico_City...,2024-10-25,18:30:00,2024-10-25,22:00:00,2024-10-26,17:30:00,2024-10-26,21:00:00,\N,\N
1121,1141,2024,21,18,São Paulo Grand Prix,2024-11-03,17:00:00,https://en.wikipedia.org/wiki/2024_S%C3%A3o_Pa...,2024-11-01,14:30:00,2024-11-01,18:30:00,\N,\N,2024-11-02,18:00:00,2024-11-02,14:00:00
1122,1142,2024,22,80,Las Vegas Grand Prix,2024-11-23,06:00:00,https://en.wikipedia.org/wiki/2024_Las_Vegas_G...,2024-11-21,02:30:00,2024-11-21,06:00:00,2024-11-22,02:30:00,2024-11-22,06:00:00,\N,\N
1123,1143,2024,23,78,Qatar Grand Prix,2024-12-01,17:00:00,https://en.wikipedia.org/wiki/2024_Qatar_Grand...,2024-11-29,13:30:00,2024-11-29,17:30:00,\N,\N,2024-11-30,17:00:00,2024-11-30,13:00:00


# 2 - Extracción de las condiciones meteorologicas mediante URL (html)

In [3]:
weather_races_id = []
weather_races_names = []
weather_races_year = []
weather_description = []

### Beautifulsoup

In [4]:
filtro_1984 = df_races["year"] >= 1984
races_1984 = df_races.loc[filtro_1984]

In [5]:
for index, row in df_races.iterrows():
    wiki = requests.get(df_races.iloc[index]['url']).text
    wiki_soup = bs(wiki, 'html.parser')
    
    try:
        description = wiki_soup.find('th', class_='infobox-label', text='Weather').find_next_sibling('td', class_='infobox-data').text
    except:
        description = 'No info'

    weather_races_id.append(df_races.iloc[index]['raceId'])
    weather_races_names.append(df_races.iloc[index]['name'])
    weather_races_year.append(df_races.iloc[index]['year'])
    weather_description.append(description)

C:\Users\Luis\AppData\Local\Temp\ipykernel_19760\4200422365.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description = wiki_soup.find('th', class_='infobox-label', text='Weather').find_next_sibling('td', class_='infobox-data').text


### En algunos casos la descripcion de las condiciones climaticas es una lista dentro de una lista.  

In [6]:

for i in range(len(weather_description)):
    if type(weather_description[i]) == str:
        weather_description[i] = weather_description[i].replace('\n', '')
    
    if type(weather_description[i]) == list:
        weather_description[i] = weather_description[i][0]

### Se crea el data frame con las listas anteriores y se guarda el data frame en formato .csv

In [7]:
df_weather = pd.DataFrame({'year': weather_races_year, 'name': weather_races_names, 'race_id': weather_races_id, 'weather_description': weather_description}).sort_values(by=['year'])
df_weather = df_weather.drop_duplicates(subset=['race_id'])

In [8]:
df_weather.to_csv("./data/weather_description_bs.csv")

# Lectura de las condiciones y transformacion en WET o DRY

### Creación de columna con las descripciones del tiempo

In [9]:
wet_races = df_weather

In [10]:
wet_races = wet_races.reset_index(drop=True)

In [11]:
wet_races

,year,name,race_id,weather_description
0,1950,Italian Grand Prix,839,"Warm, hot and sunny"
1,1950,French Grand Prix,838,Hot and sunny
2,1950,Belgian Grand Prix,837,"Warm, dry and sunny"
3,1950,Swiss Grand Prix,836,"Warm, dry and sunny"
4,1950,Indianapolis 500,835,No info
...,...,...,...,...
1120,2024,Australian Grand Prix,1123,Sunny
1121,2024,Saudi Arabian Grand Prix,1122,Clear
1122,2024,Bahrain Grand Prix,1121,Clear
1123,2024,Austrian Grand Prix,1131,Partly cloudy


### Creación de lista donde se almacenara el resultado de la busqueda de las palabras clave que nos diran si la carrera es o no WET

In [12]:
weather = []

for index, row in wet_races.iterrows():
    r = re.compile('rain|wet|rainy|typhoon|monsoon|shower|showers|thunderstorm|thunderstorms|drizzle|drizzles', re.IGNORECASE)
    count = len(r.findall(wet_races.iloc[index]['weather_description']))
    
    if count >= 1:
        weather.append('Wet')
    elif count == 0:
        weather.append('Dry')

### Creación de la columna "weather" donde se añadira el resultado obtenido

In [13]:
wet_races['weather'] = weather

In [14]:
wet_races

,year,name,race_id,weather_description,weather
0,1950,Italian Grand Prix,839,"Warm, hot and sunny",Dry
1,1950,French Grand Prix,838,Hot and sunny,Dry
2,1950,Belgian Grand Prix,837,"Warm, dry and sunny",Dry
3,1950,Swiss Grand Prix,836,"Warm, dry and sunny",Dry
4,1950,Indianapolis 500,835,No info,Dry
...,...,...,...,...,...
1120,2024,Australian Grand Prix,1123,Sunny,Dry
1121,2024,Saudi Arabian Grand Prix,1122,Clear,Dry
1122,2024,Bahrain Grand Prix,1121,Clear,Dry
1123,2024,Austrian Grand Prix,1131,Partly cloudy,Dry


### Se guarda el data frame en formato .csv

In [15]:
wet_races.to_csv("./data/weather_description_bs_completed.csv")

In [18]:
wet_races

,year,name,race_id,weather_description,weather
0,1950,Italian Grand Prix,839,"Warm, hot and sunny",Dry
1,1950,French Grand Prix,838,Hot and sunny,Dry
2,1950,Belgian Grand Prix,837,"Warm, dry and sunny",Dry
3,1950,Swiss Grand Prix,836,"Warm, dry and sunny",Dry
4,1950,Indianapolis 500,835,No info,Dry
...,...,...,...,...,...
1120,2024,Australian Grand Prix,1123,Sunny,Dry
1121,2024,Saudi Arabian Grand Prix,1122,Clear,Dry
1122,2024,Bahrain Grand Prix,1121,Clear,Dry
1123,2024,Austrian Grand Prix,1131,Partly cloudy,Dry
